# Resolvendo o Wine Quality com Random Forest (RF)

Exemplo simples resolvendo o dataset Wine Quality com um classificador Random Forest (RF).

## Imports

In [1]:
import os
from pathlib import Path

import mlflow
import numpy as np
import pandas as pd
import plotly.express as px
from sklearn import datasets
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

# Ao definirmos os valores para essas variáveis de ambiente,
#   conseguimos acessar os dados guardados no servidor remoto!
os.environ['MLFLOW_TRACKING_URI']= "http://<URL>:8080"
os.environ['MLFLOW_TRACKING_USERNAME']= "<USER>"
os.environ['MLFLOW_TRACKING_PASSWORD']= "<PASSWORD>"

## Parâmetros

In [2]:
# Primeiro, precisamos marcar essa célula com a tag "parameters"
# https://papermill.readthedocs.io/en/latest/usage-parameterize.html

experiment_name: str = 'tutorials/example/wine_quality'
run_name: str = 'rf_1'

n_estimators: int = 10
criterion: str = 'gini'
max_depth: int | None = None
bootstrap: bool = False
oob_score: bool = False
random_state = 27894018
verbose: int = 0

test_size: float = 0.3

## Funções e Classes Utilitárias

In [3]:
def confusion_matrix_fig(confusion: np.ndarray,
                         classes: list[int] = list(range(1, 4)),
                         pred_label='Modelo',
                         true_label='Real',
                         color_label="Quantidade"):
    """Essa função retorna uma figura do Plotly representando
    a matrix de confusão das predições recebidas como entrada.

    Args:
        confusion (np.ndarray): matriz de confusão das predições.
        pred_label (str, optional): label usada para predições do modelo. Defaults to 'Modelo'.
        true_label (str, optional): label usada para valores reais. Defaults to 'Real'.
        color_label (str, optional): label usada para definir as cores. Defaults to "Quantidade".

    Returns:
        fig: figura do Plotly.
    """
    fig = px.imshow(confusion,
                    labels=dict(x=pred_label,
                                y=true_label,
                                color=color_label),
                    x=classes,
                    y=classes,
                    text_auto=True)
    fig.update_xaxes(side="top")
    fig.update_layout(xaxis={
        'tickmode': 'array',
        'tickvals': classes,
    })

    return fig


def feature_importance_fig(etree,
                           features_names: list[str]):
    importance = etree.feature_importances_
    std = np.std([tree.feature_importances_ for tree in etree.estimators_], axis=0)
    etree_importance = pd.DataFrame({
        'Feature': features_names,
        'Average Gini Importance': importance,
        'std': std
    })

    fig = px.bar(etree_importance,
                 x='Feature',
                 y='Average Gini Importance',
                 error_y='std')
    return fig

## Dataset

In [4]:
# Carregando o dataset
wine = datasets.load_wine()
features = wine.data
target = wine.target

# Preparando os splits de treino e teste
X_train, X_test, y_train, y_test = train_test_split(features,
                                                    target,
                                                    test_size=test_size,
                                                    stratify=target,
                                                    random_state=random_state)

# Criando datasets de treino e teste do mlflow
ds_train = mlflow.data.from_numpy(X_train, targets=y_train)
ds_test = mlflow.data.from_numpy(X_test, targets=y_test)

## Experimento

In [5]:
# Vamos criar um experimento com esse nome caso ele não exista
# Primeiro, obtemos uma lista de experimentos com esse nome
experiments = mlflow.search_experiments(
    filter_string=f"name = '{experiment_name}'")

# Caso não tenham sido encontrados experimentos, precisamos criar um novo
if len(experiments) <= 0:
    experiment_id = mlflow.create_experiment(name=experiment_name)
else:
    experiment_id = experiments[0].experiment_id

# Vamos iniciar uma nova run para armazenar os resultados
with mlflow.start_run(experiment_id=experiment_id,
                      run_name=run_name) as run:

    # Realizando o log de parâmetros e
    #  hiper-parâmetros
    mlflow.log_params({
        'n_estimators': n_estimators,
        'criterion': criterion,
        'max_depth': max_depth,
        'bootstrap': bootstrap,
        'oob_score': oob_score,
        'random_state': random_state,
        'verbose': verbose,
        'train_samples': len(X_train),
        'test_samples': len(X_test)
    })

    # Instanciando o classificador
    clf = ExtraTreesClassifier(n_estimators=n_estimators,
                               criterion=criterion,
                               max_depth=max_depth,
                               bootstrap=bootstrap,
                               oob_score=oob_score,
                               random_state=random_state,
                               verbose=verbose)

    # Realizando log dos dados de treinamento
    mlflow.log_input(ds_train, context="training")

    # Realizando treinamento do classificador
    clf.fit(ds_train.features, ds_train.targets)

    # Salvando modelo
    mlflow.sklearn.log_model(clf, "extra_trees")

    # Realizando log dos dados de treinamento
    mlflow.log_input(ds_test, context="testing")

    # Realizando predições para obter as métricas
    y_pred = clf.predict(ds_test.features)

    # Geração das métricas
    cm = confusion_matrix(ds_test.targets, y_pred)
    report = classification_report(y_test,
                                   y_pred,
                                   digits=3,
                                   output_dict=True,
                                   zero_division=0)

    # Salvando as métricas de classificação
    for k in report['weighted avg']:
        mlflow.log_metric(k,
                          report['weighted avg'][k])

    # Obtenção da matriz de confusão
    cm_fig = confusion_matrix_fig(cm)

    # Salvando imagem
    cm_fig.write_image('confusion_matrix.png')

    # Salvando o artefato
    mlflow.log_artifact('confusion_matrix.png')

    # Obtenção da figura de importância das características
    fi_fig = feature_importance_fig(clf,
                                    wine.feature_names)

    # Salvando imagem
    fi_fig.write_image('feature_importance.png')

    # Salvando o artefato
    mlflow.log_artifact('feature_importance.png')

    # Removendo arquivos
    os.remove('confusion_matrix.png')
    os.remove('feature_importance.png')